# Train a Classifier to detect traffic lights

##### Udacity Self-Driving Car Nanodegree
##### project system integration
##### team vulture

This approach follows very closely the one documented in https://github.com/cena0805/ros-traffic-light-classifier as this is almost what we need.

Rainer Bareiss, V0.1, August 20th, 2017, Lago Maggiore

## 1 imports

In [62]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD
from sklearn.model_selection import train_test_split
#from visual_callbacks import AccLossPlotter
import model
import numpy as np
import glob
import os

## 2 parameters

In [63]:
batch_size = 64
nb_epoch = 3 #25
nb_train_samples = 200

## 3 load & explore model

In [64]:
# original model from https://github.com/cena0805/ros-traffic-light-classifier
model = model.get_model()

In [65]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_21 (Convolution2D) (None, 64, 64, 32)    896         convolution2d_input_6[0][0]      
____________________________________________________________________________________________________
activation_31 (Activation)       (None, 64, 64, 32)    0           convolution2d_21[0][0]           
____________________________________________________________________________________________________
convolution2d_22 (Convolution2D) (None, 62, 62, 32)    9248        activation_31[0][0]              
____________________________________________________________________________________________________
activation_32 (Activation)       (None, 62, 62, 32)    0           convolution2d_22[0][0]           
___________________________________________________________________________________________

In [143]:
# TODO: iplement squeezenet and compare accuracy & runtime for small data set

In [144]:
# ...

## 4 load data and split into training, validate and test data

In [66]:
mydir = '/Users/rainerbareiss/Library/Mobile Documents/M6HJR9W95L~com~textasticapp~textastic/Documents/02-Udacity/carND/03-term3/P3-CarND-Capstone/90-sample_classifier/model'
myfiles = glob.glob(mydir+"/images/*/*.*")
#print(myfiles)
train_samples, validation_samples = train_test_split(myfiles, test_size=0.2)
print(len(train_samples))
print(len(validation_samples))

34972
8744


## 5 prepare data fro training 

### input

/images:
    /red
    /green
    /unknown
    
### output (augmented)

/train:
    /red
    /green
    /unknown
    
/valid:
    /red
    /green
    /unknown

In [67]:
datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.1,
                             fill_mode='nearest', rescale=1. / 255)


In [68]:
image_data_gen = datagen.flow_from_directory('images', target_size=(64, 64), classes=['green', 'red', 'unknown'],
                                             batch_size=batch_size)
print("number of image samples: ",image_data_gen.nb_sample)
print("using samples:           ",nb_train_samples)

Found 43716 images belonging to 3 classes.
number of image samples:  43716
using samples:            200


In [81]:
# https://stackoverflow.com/questions/3397157/how-to-read-a-raw-image-using-pil
from PIL import Image as Image

mydir_in = '/Users/rainerbareiss/Library/Mobile Documents/M6HJR9W95L~com~textasticapp~textastic/Documents/02-Udacity/carND/03-term3/P3-CarND-Capstone/00-team_vulture/SDC-System-Integration/classifier'
# read image
i = Image.open(mydir_in+'/images/red/red_123.jpg')
imgSize = i.size
rawData = i.tobytes()

# save in deifferent formats
#img = Image.frombytes('L', imgSize, rawData)
#img.save('output/lmode.png')
img = Image.frombytes('RGB', imgSize, rawData)
img.save('output/red_123.jpg')
#img = Image.frombytes('RGBX', imgSize, rawData)
#img.save('rgbxmode.jfif')
#img = Image.frombytes('RGBA', imgSize, rawData)
#img.save('rgbamode.png')
#img = Image.frombytes('CMYK', imgSize, rawData)
#img.save('rgbamode.tiff')

### store training data into separate folder

In [135]:
# https://stackoverflow.com/questions/3397157/how-to-read-a-raw-image-using-pil

# input & output folder
mydir_in = '/Users/rainerbareiss/Library/Mobile Documents/M6HJR9W95L~com~textasticapp~textastic/Documents/02-Udacity/carND/03-term3/P3-CarND-Capstone/00-team_vulture/SDC-System-Integration/classifier/'
mydir_out = '/Users/rainerbareiss/Downloads/traffic_light_images/'

In [130]:
for ifile in train_samples:
    # read data
    file = Image.open(ifile)
    imgSize = file.size
    rawData = file.tobytes()
    
    # get class & file name
    classdir = ifile.split("/")[-2]   # class name
    classname = ifile.split("/")[-1]  # name of the image file
    #print(classdir + ' ---> ' + classname)
    
    # generate new file names
    ofile = mydir_out + 'train/' + classdir + '/' + classname
    #print(ofile)
    
    # save in new folder
    img = Image.frombytes('RGB', imgSize, rawData)
    img.save(ofile)
    #print(ifile)

In [134]:
datagen = ImageDataGenerator(width_shift_range=.2, height_shift_range=.2, shear_range=0.05, zoom_range=.1,
                             fill_mode='nearest', rescale=1. / 255)
train_image_data_gen = datagen.flow_from_directory(mydir_out+'train', target_size=(64, 64), classes=['green', 'red', 'unknown'],
                                             batch_size=batch_size)
print("number of image samples: ",train_image_data_gen.nb_sample)

Found 34972 images belonging to 3 classes.
number of image samples:  34972


### store validation data into separate folder

In [139]:
for ifile in validation_samples:
    # read data
    file = Image.open(ifile)
    imgSize = file.size
    rawData = file.tobytes()
    
    # get class & file name
    classdir = ifile.split("/")[-2]   # class name
    classname = ifile.split("/")[-1]  # name of the image file
    #print(classdir + ' ---> ' + classname)
    
    # generate new file names
    ofile = mydir_out + 'valid/' + classdir + '/' + classname
    #print(ofile)
    
    # save in new folder
    img = Image.frombytes('RGB', imgSize, rawData)
    img.save(ofile)
    #print(ifile)

In [141]:
valid_image_data_gen = datagen.flow_from_directory(mydir_out+'valid', target_size=(64, 64), classes=['green', 'red', 'unknown'],
                                             batch_size=batch_size)
print("number of image samples: ",valid_image_data_gen.nb_sample)

Found 8744 images belonging to 3 classes.
number of image samples:  8744


## 6 train model

In [142]:
model.fit_generator(train_image_data_gen, nb_epoch=nb_epoch, samples_per_epoch=nb_train_samples)

Epoch 1/3
256/200 [======================================] - 6s - loss: 0.8964 - acc: 0.6172

/Users/rainerbareiss/Developer/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1573: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '



Epoch 2/3
256/200 [======================================] - 5s - loss: 0.7700 - acc: 0.7266
Epoch 3/3
256/200 [======================================] - 5s - loss: 0.7766 - acc: 0.7227


## 7 save model

In [70]:
model.save('traffic_lights_classifier.h5')

## 8 predict 

In [146]:
import os
from PIL import Image
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

def reshape_image(image):
  x = img_to_array(image.resize((64, 64), Image.ANTIALIAS))
  return x[None, :]

model_predict = load_model('traffic_lights_classifier.h5')

In [156]:
# Load Example Image
green = load_img(mydir_in + 'images_from_simulator/sampleout1.jpg')

# TODO: scale & crop before prediction!!!
print(model_predict.predict(reshape_image(green)))

[[ 0.  0.  1.]]
